In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
%cd /content/drive/MyDrive/Colab_Notebooks/CV/CoursePro-2/

/content/drive/MyDrive/Colab_Notebooks/CV/CoursePro-2


In [14]:
root = '/content/drive/MyDrive/Colab_Notebooks/CV/CoursePro-2/'

In [15]:
image_folder = '/content/drive/MyDrive/Colab_Notebooks/CV/CoursePro-2/images'

In [16]:
import cv2
import numpy as np
import os
import json

In [17]:
with open(os.path.join(root, "groundtruth.json"), 'r') as file:
    json_data = file.read()
dictionary = json.loads(json_data)

In [19]:
import itertools
keys = dictionary.keys()
target_img = []
query_img = []
for key in itertools.islice(keys, 50):
    value = dictionary[key]
    query_image = cv2.imread( os.path.join(image_folder, value['query']), cv2.IMREAD_GRAYSCALE)
    query_image = cv2.resize(query_image, (128, 128))
    query_img.append(query_image)
    target_img.append(value['similar'])

In [20]:
# Create ORB object
orb = cv2.ORB_create()

In [21]:
final_result = {}

for i, query_image in enumerate(query_img):
    # Detect keypoints and compute descriptors for the query image
    kp_target, des_target = orb.detectAndCompute(query_image, None)
    
    top_matched_images = []
    
    for filename in os.listdir('./images'):
        # Load the image from file and convert it to grayscale
        img_path = os.path.join('./images', filename)
        img_data = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (128, 128))
        
        # Detect keypoints and compute descriptors for the current image
        kp, des = orb.detectAndCompute(img_data, None)
        
        # Check if keypoints and descriptors are valid
        if kp is None or des is None:
            continue
        
        # Check if descriptors have the same data type
        if des.dtype != des_target.dtype:
            des = des.astype(des_target.dtype)
        
        # Convert descriptors to compatible format
        des = np.uint8(des)
        
        # Compute matches using Brute-Force matcher
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = matcher.match(des_target, des)
        
        # Convert matches to list and sort by distance
        matches = sorted(matches, key=lambda x: x.distance)
        
        # Keep the top 4 matches
        top_matches = matches[:4]
        
        # Get the number of matches
        num_matches = len(top_matches)
        
        # Check if the current image has more matches than the previous maximum
        if num_matches > 0:
            # Store the image path in the list
            top_matched_images.append(img_path)
    
    # Keep only the top 4 matched images
    top_matched_images = top_matched_images[:4]
    
    final_result[i] = top_matched_images


In [22]:
file_path = "/content/drive/MyDrive/Colab_Notebooks/CV/CoursePro-2/orb_result.json"

In [23]:
with open(file_path, 'w') as json_file:
    json.dump(final_result, json_file)

In [24]:
final_result

{0: ['./images/118300.jpg',
  './images/118802.jpg',
  './images/118003.jpg',
  './images/118302.jpg'],
 1: ['./images/118300.jpg',
  './images/118802.jpg',
  './images/118003.jpg',
  './images/118302.jpg'],
 2: ['./images/118300.jpg',
  './images/118802.jpg',
  './images/118003.jpg',
  './images/118302.jpg'],
 3: ['./images/118300.jpg',
  './images/118802.jpg',
  './images/118003.jpg',
  './images/118302.jpg'],
 4: ['./images/118300.jpg',
  './images/118802.jpg',
  './images/118003.jpg',
  './images/118302.jpg'],
 5: ['./images/118300.jpg',
  './images/118802.jpg',
  './images/118003.jpg',
  './images/118302.jpg'],
 6: ['./images/118300.jpg',
  './images/118802.jpg',
  './images/118003.jpg',
  './images/118302.jpg'],
 7: ['./images/118300.jpg',
  './images/118802.jpg',
  './images/118003.jpg',
  './images/118302.jpg'],
 8: ['./images/118300.jpg',
  './images/118802.jpg',
  './images/118003.jpg',
  './images/118302.jpg'],
 9: ['./images/118300.jpg',
  './images/118802.jpg',
  './images